No olvidar eliminar nbqa y pydocstyle y git

ver lo de las comillas en out_samples

In [112]:
!nbqa pydocstyle --select=D107,D400,D401,D413 parse_metadata.ipynb

# parse_metadata

input (2):  Suplementary_Table_1.csv y archivos

output (2): samples.tsv, labels.tsv

In [89]:
import os
import pandas as pd
import numpy as np
from glob import glob

def normalize_species(specie):
    """Normaliza el nombre de especies biológicas estandarizando prefijos y caracteres.

    Args:
        specie (str): Nombre de la especie a normalizar (ej: "bifidobacterium_longum").

    Returns:
        str: Nombre normalizado con prefijo 's__', espacios en lugar de guiones/barras bajas,
             y capitalización (ej: "s__Bifidobacterium longum").

    Examples:
        >>> normalize_species("bifidobacterium_longum")
        's__Bifidobacterium longum'
        >>> normalize_species("s__bifidobacterium-longum")
        's__Bifidobacterium longum'
    """
    if not specie.startswith('s__'):
        specie = 's__' + specie.replace('_', ' ').replace('-', ' ').capitalize()
    else:
        specie = 's__' + specie[3:].replace('_', ' ').replace('-', ' ').capitalize()
    return specie

def read_fasta(x, Dictionary=False):
    """
    Lee un archivo FASTA y retorna su contenido junto con la información de las secuencias.

    Args:
        x (str): Ruta al archivo FASTA.
        Dictionary (bool, opcional): Si es True, retorna un diccionario con metadatos de las secuencias.
            Si es False, retorna el contenido bruto y un DataFrame con la información de las secuencias.
            Por defecto es False.

    Returns:
        tuple:
            - str: Contenido bruto del archivo FASTA.
            - pandas.DataFrame o dict: DataFrame con metadatos de las secuencias si Dictionary es False,
              de lo contrario un diccionario con los metadatos de las secuencias.

    Ejemplo:
        >>> contenido, info_secuencias = read_fasta("genoma.fna")
        >>> contenido, dicc_secuencias = read_fasta("genoma.fna", Dictionary=True)
    """
    with open(x, 'r') as f:
        lectura = f.read()
    dictionary = {}
    starts, new_lines = find('>', lectura), find('\n', lectura)
    if len(starts) == 1:
        en = 0
        lectura = lectura[starts[en]:new_lines[en]+1] + lectura[new_lines[en]+1:].replace('\n', '')
        start_name, end_name, start_seq, end_seq = starts[en], new_lines[en*2], new_lines[en*2]+1, new_lines[-1]
        dictionary[en] = {'start_name': start_name,
                          'end_name': end_name,
                          'len_name': end_name-start_name,
                          'name': lectura[start_name:end_name],
                          'start_seq': start_seq,
                          'end_seq': end_seq,
                          'len_seq': end_seq-start_seq,
                          'seq': lectura[start_seq:end_seq]}
    else:
        # secuencias
        # clave = (>.*\n)
        # valor = lectura[(key[-1].index+1)].*\n #se considera que cada valor está entre \n
        for en in range(len(starts)):
            start_name, end_name, start_seq, end_seq = starts[en], new_lines[en*2], new_lines[en*2]+1, new_lines[en*2+1]
            dictionary[en] = {'start_name': start_name,
                              'end_name': end_name,
                              'len_name': end_name-start_name,
                              'name': lectura[start_name:end_name],
                              'start_seq': start_seq,
                              'end_seq': end_seq,
                              'len_seq': end_seq-start_seq,
                              'seq': lectura[start_seq:end_seq]}
    if not Dictionary:
        return lectura, pd.DataFrame.from_dict(dictionary, orient='index').astype({
            'start_name': 'uint32',
            'end_name': 'uint32',
            'len_name': 'uint32',
            'name': object,
            'start_seq': 'uint32',
            'end_seq': 'uint32',
            'len_seq': 'uint32',
            'seq': object})
    return lectura, dictionary

def find(subcadena, texto):
    """
    Busca todas las posiciones de una subcadena dentro de un texto.

    Args:
        subcadena (str): Subcadena a buscar en el texto.
        texto (str): Texto donde se realiza la búsqueda.

    Returns:
        list: Lista de posiciones (índices) donde se encuentra la subcadena en el texto.

    Ejemplo:
        >>> find('a', 'banana')
        [1, 3, 5]
    """
    posiciones = []
    indice = 0
    while True:
        indice = texto.find(subcadena, indice)
        if indice == -1:
            break
        posiciones.append(indice)
        indice += 1
    return posiciones

def result(csv_table, files_by_table, fasta_ext, species):
    # data normalizada
    data = pd.read_csv(csv_table, low_memory=False) # leyendo csv_table
    data['normalize_species'] = data[['species']].map(normalize_species)

    # especies normalizadas
    norm_species = [normalize_species(ii) for ii in species]

    # ficheros de interés por especies
    fasta_files = [os.path.split(ii)[-1] for ii in glob(files_by_table+f'/*.{fasta_ext}')]
    dict_genomas = {species[en]: np.unique([ii for ii in data[data['normalize_species'] == esp]['accession']]) for en,esp in enumerate(norm_species)}
    dict_genomas = {clave: [file for file in fasta_files for ii in dict_genomas[clave] if file.startswith(ii)] for clave in dict_genomas}

    samples = [list(ii) for ii in dict_genomas.values()]
    labels = list(dict_genomas.keys())

    # tsv output
    OUTPUT='../Results/parse_metadata'
    os.makedirs(OUTPUT, exist_ok=True)
    out_labels = pd.DataFrame(labels, columns=['species'])
    out_labels.to_csv(f'{OUTPUT}/labels.tsv', index=False, header=False)
    out_samples = pd.DataFrame([samples])
    out_samples.to_csv(f'{OUTPUT}/samples.tsv', index=False, header=False)

    return out_labels, out_samples

In [152]:
# especies
species = ["Bifidobacterium longum", "Escherichia fergusonii", "Bacillus_A cereus", "Xylella fastidiosa"]

# rutas
csv_table = "/data/Proyecto/Supplememtary_Table_1.csv"
files_by_table = "/data/Proyecto/Data_forTAP"
fasta_ext = 'fna'
#------------------------------------------------------------

# data normalizada
data = pd.read_csv(csv_table, low_memory=False) # leyendo csv_table
data['normalize_species'] = data[['species']].map(normalize_species)

# especies normalizadas
norm_species = [normalize_species(ii) for ii in species]

# ficheros de interés por especies
fasta_files = [os.path.split(ii)[-1] for ii in glob(files_by_table+f'/*.{fasta_ext}')] # todos los ficheros fasta
labels = {}
for en, esp_norm in enumerate(norm_species):
    selection = data[data['normalize_species'] == esp_norm][['accession', 'checkm_marker_lineage',
                                                             'ssu_silva_blast_subject_id',
                                                             'ssu_silva_taxonomy', 'species']]
    selection['filename'] = selection['accession'].apply(
        lambda acc: next((f for f in fasta_files if f.startswith(acc)), None)
    )
    # Reordenar para que 'filename' sea la primera columna
    cols = ['filename'] + [col for col in selection.columns if col != 'filename']
    labels[species[en]] = selection[cols]

# dict_genomas_short = {species[en]: \
#                 np.unique([ii for ii in data[data['normalize_species'] == esp]['accession']]) \
#                 for en,esp in enumerate(norm_species)} # el empezar del nombre
# dict_genomas = {clave: [file for file in fasta_files for ii in dict_genomas_short[clave] \
#                 if file.startswith(ii)] for clave in dict_genomas_short} # el nombre completo

In [158]:
pd.DataFrame.from_dict(labels, orient='index')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.